In [ ]:
#!/usr/bin/env python
# coding: utf-8

# # ASSIGNMENT 3

# __Anselmo Raffaele__
# 
# matricola __846842__
# 
# r.anselmo@campus.unimib.it
# 
# 

# ## Data Import and Preprocessing

# In[1]:


import numpy as np
import keras
import matplotlib as mpl
import matplotlib.pyplot as plt 
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from keras.utils import to_categorical
from keras.optimizers import adam


# In[2]:


from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()


# In[3]:


plt.imshow(x_train[0],cmap='Greys' )


# In[4]:


x_train[0].shape


# In[5]:


from collections import Counter
count = Counter(y_train)
count


# In[6]:


# Reshaping the array to 4-dims so that it can work with the Keras API
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)
# Making sure that the values are float so that we can get decimal points after division
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# Normalizing the RGB codes by dividing it to the max RGB value.
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print('Number of images in x_train', x_train.shape[0])
print('Number of images in x_test', x_test.shape[0])


# In[7]:


#one-hot encode target column
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


# ### Train-Val Split

# Malgrado nella consegna fosse stato indicato di non utilizzare nessuna ulteriore partizione del dataset, questa si è rivelata necessarioa in quanto il MNIST dataset non conteneva alcun Validation set.

# In[8]:


from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train,test_size=0.2)


# ### Defining *plot_metrics* function
# 
# La seguente funzione sarà utilizzata per visualizzare le performance dei modelli sulla base dell'andamento di **loss** e **accuracy** nella progressione dell'apprendimento durante le differenti epoche

# In[9]:


mpl.rcParams['figure.figsize'] = (18, 12)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']


# In[10]:


#Accuracy and Loss plot function
def plot_metrics(history):
  metrics =  ['loss', 'accuracy']
  for n, metric in enumerate(metrics):
    name = metric.replace("_"," ").capitalize()
    plt.subplot(2,2,n+1)
    plt.plot(history.epoch,  history.history[metric], 
             color=colors[0],
             label='Train')
    plt.plot(history.epoch, history.history['val_'+metric],
             color=colors[0],
             linestyle="--", label='Val')
    plt.xlabel('Epoch')
    plt.ylabel(name)
    if metric == 'loss':
      plt.ylim([0, plt.ylim()[1]])
    elif metric == 'accuracy':
      plt.ylim([0.2,1])
    else:
      plt.ylim([0,1])

    plt.legend()


# ## Convolutional Neural Network Model

# In[11]:


#defining some hyper-parameters
n_epochs = 10
batch_size = 1024
METRICS = [ 
      ['accuracy']
]


# In[12]:


# Creating a Sequential Model and adding the layers
# Creating a Sequential Model and adding the layers
def make_model(metrics= METRICS):
 model = Sequential([
        Conv2D(16, kernel_size=(3), activation = 'relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(16, kernel_size=(3), activation = 'relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(), # Flattening the 2D arrays for fully connected layers
        Dense(12, activation='relu'),
        Dropout(0.2),
        Dense(10,activation='softmax')       
 ])
    
 model.compile(optimizer= keras.optimizers.Adam(lr=1e-3), 
              loss='categorical_crossentropy', 
              metrics=METRICS)
    
 return model


# In[13]:


model = make_model()


# In[14]:


model.summary()


# In[15]:


model_history = model.fit(x_train,
                          y_train, 
                          batch_size=batch_size,
                          epochs=n_epochs,
                          validation_data=(x_val, y_val))


# In[16]:


plot_metrics(model_history)


# In[17]:


score = model.evaluate(x_test, y_test, verbose=0)
print('Test Loss:', score[0])
print('Test Accuracy:', score[1])

